# This program will generate labels for the entire API_specific expert and sub_expert columns
The processes will all match small_LLM. Make sure to test with small_LLM to ensure that the process will work smoothly, as running all the calls on a large dataset cost a lot. If any changes need to be made in small_LLM, make sure they are made here exactly the same. The only difference should be the lenth of the dataframe passed in (and less practice code).

In [27]:
import json
import pandas as pd
import os

### Load domain and subdomain labels

In [28]:
with open('./data/domain_labels.json', 'r') as f:
    domain_json = json.load(f) 
with open('./data/subdomain_labels.json', 'r') as f:
    subdomain_json = json.load(f)

In [29]:
def create_domains_list(domain_json):
    domains = []
    items_list = domain_json["Items"]
    
    for i in range(len(items_list)):
        domains.append(list(items_list[i].keys())[0]) #looks strange because the data type keeps swapping dict and list.
        
    return domains

In [30]:
def create_subdomains_dict(subdomain_json, domains):
    subdomains = {}
    
    for domain in domains:
        items_list = subdomain_json[domain]
        subdomain_list = []
        
        for i in range(len(items_list)):
            subdomain_list.append(list(items_list[i].keys())[0])
            
        subdomains.update({domain : subdomain_list})
    
    return subdomains

In [31]:
domains = create_domains_list(domain_json)
subdomains = create_subdomains_dict(subdomain_json, domains)

In [32]:
print(domains)

['Application', 'Application Performance Manager', 'Big Data', 'Cloud', 'Computer Graphics', 'Data Structure', 'Databases', 'DevOps', 'Error Handling', 'Event Handling', 'Geographic Information System', 'Input-Output', 'Interpreter', 'Internationalization', 'Logic', 'Language', 'Logging', 'Machine Learning', 'Microservices/Services', 'Multimedia', 'Multi-Thread', 'Natural Language Processing', 'Network', 'Operating System', 'Parser', 'Search', 'Security', 'Setup', 'User Interface', 'Utility', 'Test']


In [33]:
for domain in domains:
    print(("Domain: " + domain + '\nSubdomains(%d): ' + ', '.join(subdomains[domain]) + '\n') % (len(subdomains[domain])))

Domain: Application
Subdomains(6): Integration, Plugin Management, User Customization, App Configuration, Version Control, Compatibility Checks

Domain: Application Performance Manager
Subdomains(6): Performance Monitoring, Resource Allocation, Error Detection, Load Balancing, Traffic Management, Diagnostic Tools

Domain: Big Data
Subdomains(6): Data Processing, Data Storage, Data Analysis, Real-Time Processing, Batch Processing, Data Visualization

Domain: Cloud
Subdomains(6): Resource Management, Virtualization, Scalability Solutions, Cloud Security, Data Migration, Service Configuration

Domain: Computer Graphics
Subdomains(6): Image Rendering, Animation, Modeling, Texture Mapping, Visual Effects, Graphics Optimization

Domain: Data Structure
Subdomains(6): Linear Structures, Tree Structures, Graph Structures, Data Sorting, Search Algorithms, Data Manipulation

Domain: Databases
Subdomains(6): Query Execution, Transaction Management, Schema Design, Database Security, Backup and Reco

### Setup OpenAI API Key

In [34]:
api_key = os.getenv("OPENAI_API_KEY")

In [35]:
from langchain_openai import ChatOpenAI

In [36]:
test_llm = ChatOpenAI(temperature=0, api_key=api_key)

In [37]:
response = test_llm.invoke("Test prompt, only reply True") #if no error is thrown, api key works

In [38]:
print(response)

content='True' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 13, 'total_tokens': 14, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BowPeYm7TfcvpyE5iLVLSeAgHolew', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--e356689b-8d3e-4935-95b5-14fed335f0ef-0' usage_metadata={'input_tokens': 13, 'output_tokens': 1, 'total_tokens': 14, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [39]:
print(response.content)

True


### Generate Promts

In [40]:
domain_base_prompt = "Categorize the library \"%s\" into only one of the domains, without explaining the result: "
domain_prompt = domain_base_prompt + ", ".join(domains)
print(domain_prompt % "example.library")

Categorize the library "example.library" into only one of the domains, without explaining the result: Application, Application Performance Manager, Big Data, Cloud, Computer Graphics, Data Structure, Databases, DevOps, Error Handling, Event Handling, Geographic Information System, Input-Output, Interpreter, Internationalization, Logic, Language, Logging, Machine Learning, Microservices/Services, Multimedia, Multi-Thread, Natural Language Processing, Network, Operating System, Parser, Search, Security, Setup, User Interface, Utility, Test


In [41]:
def generate_subdomain_prompt(base_prompt, domain):
    if domain not in domains: 
        #Error check should be performed outside this function but this will minimize cost if something goes wrong.
        return "Only reply PromptError"
    
    prompt = base_prompt + ", ".join(subdomains[domain])
    return prompt % (domain, "%s")

In [42]:
subdomain_base_prompt = "Given that the domain is \"%s\", categorize the library \"%s\" into one of the subdomains, without explaining the result: "
print(generate_subdomain_prompt(subdomain_base_prompt, "Internationalization") % "example.library")

Given that the domain is "Internationalization", categorize the library "example.library" into one of the subdomains, without explaining the result: Localization, Language Support, Cultural Adaptation, Currency Conversion, Date/Time Formatting, Right-to-Left Layouts


### Setup dataframe (from API_specific export)

In [43]:
columns = ['general', 'specific', 'api_name_fk', 'expert', 'sub_expert']
API_specific = pd.read_csv("./data/input/API_specific.csv", header="infer", names=columns)

In [44]:
print(API_specific.shape)
API_specific.head(10)

(2531, 5)


,general,specific,api_name_fk,expert,sub_expert
0,r,r,r.fileformat,NaN,NaN
1,r,r,r.util,NaN,NaN
2,r,fetcher,r.fetcher.citation.semanticscholar,NaN,NaN
3,r,r,r.fetcher,NaN,NaN
4,r,fetcher,r.fetcher.citation,NaN,NaN
5,r,fileformat,r.fileformat.pdf,NaN,NaN
6,r,fetcher,r.fetcher.isbntobibtex,NaN,NaN
7,r,fetcher,r.fetcher.transformers,NaN,NaN
8,r,r,r.plaincitation,NaN,NaN
9,io,io,java.io.File,NaN,NaN


### LLM Calls

In [45]:
llm = ChatOpenAI(model = "gpt-4o-mini-2024-07-18", temperature=0, api_key=api_key)

In [46]:
info = {"overall_tokens": 0, "errors": 0} #using dict gets around global variable issues

def domain_call(api_name, printing):
    prompt = domain_prompt % api_name
    response = llm.invoke(prompt)
    
    if(printing):
        meta_data = response.usage_metadata
        #print("prompt: " + prompt)
        print("api name: " + api_name)
        print("label: " + response.content)
        print("output tokens: " + str(meta_data['output_tokens']))
        print("total tokens: " + str(meta_data['total_tokens']))
        print()
        
    info['overall_tokens'] += response.usage_metadata['total_tokens']
    
    return response.content


def subdomain_call(api_name, domain, printing):
    if domain not in domains:
        info['errors'] += 1
        print("DOMAIN NOT FOUND. api: " + api_name + "domain: " + domain)
        return "DomainError"
    
    subdomain_prompt = generate_subdomain_prompt(subdomain_base_prompt, domain)
    prompt = subdomain_prompt % api_name
    response = llm.invoke(prompt)
    
    if(printing):
        meta_data = response.usage_metadata
        #print("prompt: " + prompt)
        print("api name: " + api_name)
        print("domain: " + domain)
        print("subdomain: " + response.content)
        print("output tokens: " + str(meta_data['output_tokens']))
        print("total tokens: " + str(meta_data['total_tokens']))
        print()
        
    info['overall_tokens'] += response.usage_metadata['total_tokens']
    
    return response.content


def generate_labels(input_df, printing=True):
    df = input_df.copy()
    
    info['overall_tokens'] = 0 #reset info
    info['errors'] = 0
    
    #make llm calls
    df.loc[:, 'expert'] = df['api_name_fk'].apply(domain_call, printing=printing)
    df.loc[:, 'sub_expert'] = df.apply(lambda row: subdomain_call(row['api_name_fk'], row['expert'], printing), axis=1)
    
    print("Finished.")
    print("Tokens used overall: " + str(info['overall_tokens']))
    print("Errors encountered: " + str(info['errors']))
    print("Model: " + llm.model_name)
    
    return df

In [47]:
labeled_api_specific = generate_labels(API_specific) #note: took 45 minute to run a set of 2531 * 2 = 5062 calls

api name: r.fileformat
label: Utility
output tokens: 1
total tokens: 114

api name: r.util
label: Utility
output tokens: 1
total tokens: 113

api name: r.fetcher.citation.semanticscholar
label: Search
output tokens: 1
total tokens: 121

api name: r.fetcher
label: Utility
output tokens: 1
total tokens: 114

api name: r.fetcher.citation
label: Utility
output tokens: 1
total tokens: 116

api name: r.fileformat.pdf
label: Input-Output
output tokens: 3
total tokens: 117

api name: r.fetcher.isbntobibtex
label: Utility
output tokens: 1
total tokens: 120

api name: r.fetcher.transformers
label: Utility
output tokens: 1
total tokens: 116

api name: r.plaincitation
label: Utility
output tokens: 1
total tokens: 115

api name: java.io.File
label: Input-Output
output tokens: 3
total tokens: 116

api name: java.io.IOException
label: Input-Output
output tokens: 3
total tokens: 116

api name: java.net.Authenticator
label: Network
output tokens: 1
total tokens: 115

api name: java.nio.file.Files
label

api name: org.jabref.logic.net.URLDownload
label: Network
output tokens: 1
total tokens: 119

api name: org.jabref.logic.os.OS
label: Operating System
output tokens: 2
total tokens: 119

api name: org.jabref.gui.util.DroppingMouseLocation
label: User Interface
output tokens: 2
total tokens: 123

api name: picocli.CommandLine.Option
label: Utility
output tokens: 1
total tokens: 117

api name: picocli.CommandLine.Parameters
label: Parser
output tokens: 1
total tokens: 117

api name: picocli.CommandLine.Command
label: Utility
output tokens: 1
total tokens: 117

api name: java.awt.Toolkit
label: User Interface
output tokens: 2
total tokens: 115

api name: java.awt.datatransfer.DataFlavor
label: User Interface
output tokens: 2
total tokens: 120

api name: java.awt.datatransfer.StringSelection
label: User Interface
output tokens: 2
total tokens: 120

api name: java.awt.datatransfer.Transferable
label: User Interface
output tokens: 2
total tokens: 120

api name: java.awt.datatransfer.Unsuppor

api name: javafx.scene.control.TextArea
label: User Interface
output tokens: 2
total tokens: 118

api name: javafx.scene.layout.HBox
label: User Interface
output tokens: 2
total tokens: 118

api name: javafx.scene.layout.Region
label: User Interface
output tokens: 2
total tokens: 117

api name: javafx.scene.layout.VBox
label: User Interface
output tokens: 2
total tokens: 118

api name: javafx.stage.DirectoryChooser
label: User Interface
output tokens: 2
total tokens: 117

api name: javafx.stage.FileChooser
label: User Interface
output tokens: 2
total tokens: 117

api name: javafx.stage.Window
label: User Interface
output tokens: 2
total tokens: 116

api name: javafx.util.Duration
label: User Interface
output tokens: 2
total tokens: 116

api name: org.jabref.gui.help.ErrorConsoleAction
label: Error Handling
output tokens: 2
total tokens: 121

api name: org.jabref.model.openoffice.style.CitationGroups
label: Utility
output tokens: 1
total tokens: 122

api name: org.jabref.gui.theme.Theme

api name: javafx.scene.control.ProgressIndicator
label: User Interface
output tokens: 2
total tokens: 118

api name: javafx.scene.control.Tab
label: User Interface
output tokens: 2
total tokens: 117

api name: javafx.scene.control.TabPane
label: User Interface
output tokens: 2
total tokens: 118

api name: javafx.scene.control.Tooltip
label: User Interface
output tokens: 2
total tokens: 118

api name: javafx.scene.layout.BorderPane
label: User Interface
output tokens: 2
total tokens: 118

api name: java.util.Base64
label: Utility
output tokens: 1
total tokens: 115

api name: org.jabref.gui.actions.StandardActions
label: User Interface
output tokens: 2
total tokens: 120

api name: java.util.concurrent.atomic.AtomicReference
label: Multi-Thread
output tokens: 3
total tokens: 119

api name: com.google.common.base.Strings
label: Utility
output tokens: 1
total tokens: 117

api name: org.jabref.gui.autocompleter.AutoCompletePreferences
label: User Interface
output tokens: 2
total tokens: 124


api name: dev.langchain4j.store.embedding.RelevanceScore
label: Machine Learning
output tokens: 2
total tokens: 122

api name: javafx.beans.property.ObjectProperty
label: User Interface
output tokens: 2
total tokens: 118

api name: javafx.beans.property.SimpleObjectProperty
label: User Interface
output tokens: 2
total tokens: 119

api name: javafx.beans.property.SimpleStringProperty
label: User Interface
output tokens: 2
total tokens: 119

api name: javafx.beans.property.StringProperty
label: User Interface
output tokens: 2
total tokens: 118

api name: javafx.collections.FXCollections
label: User Interface
output tokens: 2
total tokens: 118

api name: javafx.collections.ObservableMap
label: User Interface
output tokens: 2
total tokens: 117

api name: javafx.collections.transformation.FilteredList
label: User Interface
output tokens: 2
total tokens: 120

api name: org.jabref.gui.search.SearchTextField
label: User Interface
output tokens: 2
total tokens: 121

api name: org.jabref.gui.ai.

...

api name: javafx.scene.control.OverrunStyle
domain: User Interface
subdomain: Interaction Design
output tokens: 2
total tokens: 58

api name: com.google.common.cache.LoadingCache
domain: Data Structure
subdomain: Data Manipulation
output tokens: 3
total tokens: 61

api name: org.jabref.logic.importer.fileformat.ImporterTestEngine
domain: Test
subdomain: Unit Testing
output tokens: 2
total tokens: 65

api name: java.net.ServerSocket
domain: Network
subdomain: Connection Management
output tokens: 2
total tokens: 56

api name: javafx.scene.control.TitledPane
domain: User Interface
subdomain: Layout Design
output tokens: 2
total tokens: 57

api name: javafx.scene.control.TreeItem
domain: User Interface
subdomain: Interaction Design
output tokens: 2
total tokens: 56

api name: org.jabref.gui.fieldeditors.EditorTextField
domain: User Interface
subdomain: Interaction Design
output tokens: 2
total tokens: 61

api name: org.jabref.gui.fieldeditors.EditorValidator
domain: User Interface
subdomai

api name: org.jabref.logic.formatter.bibtexfields.NormalizeDateFormatter
domain: Utility
subdomain: Data Conversion
output tokens: 2
total tokens: 65

api name: org.jabref.logic.formatter.bibtexfields.NormalizeMonthFormatter
domain: Utility
subdomain: Data Conversion
output tokens: 2
total tokens: 65

api name: org.jabref.logic.formatter.bibtexfields.NormalizePagesFormatter
domain: Utility
subdomain: Data Conversion
output tokens: 2
total tokens: 65

api name: org.jabref.logic.formatter.bibtexfields.OrdinalsToSuperscriptFormatter
domain: Utility
subdomain: Data Conversion
output tokens: 2
total tokens: 70

api name: impl.org.controlsfx.skin.AutoCompletePopup
domain: User Interface
subdomain: Interaction Design
output tokens: 2
total tokens: 58

api name: org.jabref.logic.formatter.bibtexfields.UnicodeToLatexFormatter
domain: Internationalization
subdomain: Language Support
output tokens: 2
total tokens: 75

api name: javafx.event.EventHandler
domain: Event Handling
subdomain: User Inte

api name: java.security.GeneralSecurityException
domain: Security
subdomain: Encryption
output tokens: 1
total tokens: 52

api name: com.dlsc.gemsfx.TagsField
domain: User Interface
subdomain: Interaction Design
output tokens: 2
total tokens: 59

api name: com.google.common.collect.Comparators
domain: Utility
subdomain: Performance Tools
output tokens: 2
total tokens: 58

api name: org.tinylog.Logger
domain: Logging
subdomain: Event Logging
output tokens: 2
total tokens: 57

api name: org.jabref.logic.shared.DBMSConnectionProperties
domain: Databases
subdomain: Database Security
output tokens: 2
total tokens: 63

api name: org.jabref.logic.shared.DBMSConnectionPropertiesBuilder
domain: Databases
subdomain: Database Security
output tokens: 2
total tokens: 64

api name: org.jabref.logic.shared.DatabaseNotSupportedException
domain: Error Handling
subdomain: Exception Handling
output tokens: 2
total tokens: 64

api name: org.jabref.logic.shared.exception.InvalidDBMSConnectionPropertiesExce

api name: org.jabref.logic.git.GitHandler
domain: DevOps
subdomain: Version Control
output tokens: 2
total tokens: 62

api name: org.jabref.gui.autocompleter.AppendWordsStrategy
domain: User Interface
subdomain: Interaction Design
output tokens: 2
total tokens: 62

api name: org.jabref.logic.importer.fetcher.ACMPortalFetcher
domain: Utility
subdomain: Data Conversion
output tokens: 2
total tokens: 64

api name: org.jabref.logic.importer.fetcher.CompositeSearchBasedFetcher
domain: Search
subdomain: Query Processing
output tokens: 2
total tokens: 62

api name: org.jabref.logic.importer.fetcher.DBLPFetcher
domain: Search
subdomain: Query Processing
output tokens: 2
total tokens: 62

api name: org.jabref.logic.importer.fetcher.IEEE
domain: Utility
subdomain: Data Conversion
output tokens: 2
total tokens: 62

api name: org.jabref.logic.importer.fetcher.SpringerFetcher
domain: Search
subdomain: Indexing
output tokens: 2
total tokens: 61

api name: org.jabref.model.study.StudyDatabase
domain:

api name: org.jabref.gui.linkedfile.RedownloadMissingFilesAction
domain: User Interface
subdomain: Interaction Design
output tokens: 2
total tokens: 63

api name: org.jabref.gui.maintable.NewLibraryFromPdfActionOffline
domain: User Interface
subdomain: Interaction Design
output tokens: 2
total tokens: 64

api name: org.jabref.gui.maintable.NewLibraryFromPdfActionOnline
domain: User Interface
subdomain: Interaction Design
output tokens: 2
total tokens: 64

api name: org.jabref.gui.mergeentries.MergeEntriesAction
domain: User Interface
subdomain: Interaction Design
output tokens: 2
total tokens: 60

api name: org.jabref.gui.newentry.NewEntryDialogTab
domain: User Interface
subdomain: Interaction Design
output tokens: 2
total tokens: 61

api name: org.jabref.gui.preferences.ShowPreferencesAction
domain: User Interface
subdomain: Interaction Design
output tokens: 2
total tokens: 59

api name: org.jabref.gui.preview.CopyCitationAction
domain: User Interface
subdomain: Interaction Design
out

In [48]:
pd.options.display.max_seq_items = 2000
pd.options.display.max_colwidth = 90
pd.options.display.max_rows = 999

In [49]:
print(labeled_api_specific.shape)
labeled_api_specific.head(100)

(2531, 5)


,general,specific,api_name_fk,expert,sub_expert
0,r,r,r.fileformat,Utility,Data Conversion
1,r,r,r.util,Utility,Data Conversion
2,r,fetcher,r.fetcher.citation.semanticscholar,Search,Query Processing
3,r,r,r.fetcher,Utility,Data Conversion
4,r,fetcher,r.fetcher.citation,Utility,Data Conversion
5,r,fileformat,r.fileformat.pdf,Input-Output,Data Reading
6,r,fetcher,r.fetcher.isbntobibtex,Utility,Data Conversion
7,r,fetcher,r.fetcher.transformers,Utility,Data Conversion
8,r,r,r.plaincitation,Utility,Data Conversion
9,io,io,java.io.File,Input-Output,File Management


In [50]:
labeled_api_specific.to_csv("./data/output/labeled_API.csv", header=False, index=False)

Create an empty version of API_specific and import this CSV. The original can now be removed or renamed. Merging could be done but it would be more tedious.